## Содержание:
* [1. Отправка списка страниц на индексирование в Google Indexing API](#1)
* [-- Настройки](#2)
* [-- Загрузка файла xlsx с url для индексирования](#3)
* [-- Отправка на индексирование](#4)


## Пакет для работы, если не был установлен ранее

In [ ]:
#pip install google-api-python-client

# Отправка списка страниц на индексирование в Google Indexing API<a class="anchor" id="1"></a>

In [6]:
import json
import time
import requests

import httplib2
from oauth2client.service_account import ServiceAccountCredentials

import pandas as pd

## Настройки<a class="anchor" id="2"></a>

### Ресурсы для отправки

In [2]:
SCOPES = ["https://www.googleapis.com/auth/indexing"]
ENDPOINT = "https://indexing.googleapis.com/v3/urlNotifications:publish"

**параметры проекта**

In [3]:
key = 'davita-5016199-74685386f70z1b' # укажите название скачанного json файла без расширения

In [4]:
project_name = 'davita'

## Загрузка URL для индексирования<a class="anchor" id="3"></a>

Файл в формате xlsx с названием поля "urls":

In [7]:
table_with_urls_for_recrawl = pd.read_excel(f'{project_name}.xlsx')

## Отправка на индексирование <a class="anchor" id="4"></a>

Функция принимает датафрейм и ключ,<br>
отправленные адреса добавляет в отдельный список,<br>
пишет ответ и количество отправленных страниц,<br>
возвращает сформированный список.

In [8]:
def send_pages_to_google_for_recrawl(data, key):

    json_key_file = f"{key}.json"
    credentials = ServiceAccountCredentials.from_json_keyfile_name(json_key_file, 
                                                                   scopes=SCOPES)
    http = credentials.authorize(httplib2.Http())

    sent_urls_for_recrawl = []
    for url in data['urls']:
        
        urls = {'url': '{}'.format(url),
                'type': 'URL_UPDATED'}

        response, content = http.request(ENDPOINT, method="POST", 
                                        body=json.dumps(urls))

        time.sleep(1)

        if response['status'] != '200':
            break

        sent_urls_for_recrawl.append(url)

    sent_urls_list_len = len(sent_urls_for_recrawl)

    print(response['status'])
    print(content.decode())

    print('На переобход отправлено:', sent_urls_list_len)
    
    return sent_urls_for_recrawl

### Удаление из списка и экспорт

Функция принимает датафрейм и список отправленных url,<br>
Из загруженной таблицы удаляются url, которые были отправлены. Это нужно для того, чтобы в следующий раз, когда обновится квота, эти же страницы не оправлялись на переобход.

**Экспорт**

In [9]:
def export_data_to_excel(data):
    project_report = pd.ExcelWriter(f'{project_name}.xlsx', engine='xlsxwriter')
    data.to_excel(project_report, index=False)
    project_report.save()

**Удаление и экспорт**

In [10]:
def delete_sent_urls_and_export_new_table(data, sent_urls_for_recrawl):
    data = data[~data['urls'].isin(sent_urls_for_recrawl)]
    export_data_to_excel(data)

## Главная функция

In [11]:
def main_send_pages_to_google_for_recrawl(data):
    
    sent_urls_for_recrawl = send_pages_to_google_for_recrawl(data, key)
    
    delete_sent_urls_and_export_new_table(data, sent_urls_for_recrawl)

## Запуск

In [13]:
main_send_pages_to_google_for_recrawl(table_with_urls_for_recrawl)

429
{
  "error": {
    "code": 429,
    "message": "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:955122624583'.",
    "status": "RESOURCE_EXHAUSTED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.ErrorInfo",
        "reason": "RATE_LIMIT_EXCEEDED",
        "domain": "googleapis.com",
        "metadata": {
          "consumer": "projects/955122624583",
          "quota_limit_value": "200",
          "quota_metric": "indexing.googleapis.com/v3_publish_requests",
          "service": "indexing.googleapis.com",
          "quota_location": "global",
          "quota_limit": "DefaultPublishRequestsPerDayPerProject"
        }
      },
      {
        "@type": "type.googleapis.com/google.rpc.Help",
        "links": [
          {
            "description": "Request a higher quota limit.",
            "url": "https://cloud.google.com/docs/quota#requesting_highe